<a href="https://colab.research.google.com/github/pradeepprasad/Machine_Learning_Workspace/blob/master/Fruit_Quality_Check_0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below are the pre-processing steps to download data files from Kaggle and moving in colab to specific folder for persistence across multiple session.

In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
ls ../content/drive/'My Drive'/kaggle_login_file/

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp ../content/drive/'My Drive'/kaggle_login_file/kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#list datasets
!kaggle datasets list -s 'fruits'

In [0]:
cd /content/drive/My Drive/data/sample_data/

In [0]:
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification 

In [0]:
!kaggle datasets download -d moltean/fruits 

In [0]:
!mv fruits-fresh-and-rotten-for-classification.zip /content/sample_data/

In [0]:
!mv kaggle.json /content/sample_data/

In [0]:
ls ../content/sample_data/fruits-fresh-and-rotten-for-classification.zip

In [0]:
!unzip '/content/drive/My Drive/data/sample_data/fruits-fresh-and-rotten-for-classification.zip'

In [0]:
!mv dataset /content/kaggle

In [0]:
ls '/content/drive/My Drive/data/sample_data/dataset/train'

In [34]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


**Now we are proceeding to access the data available to us.**

In [23]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# TensorFlow ≥2.0-preview is required
try:
  %tensorflow_version 2.x
  print ("tensorflow Version is ", tf.__version__)
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/temp"
PROJECT_ID = "cnn/fruits_quality"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", PROJECT_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

tensorflow Version is  2.0.0


In [0]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

In [35]:
# First, we are going to load the file names and their respective target labels into numpy array! 
from sklearn.datasets import load_files
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

#train_dir = '../input/fruits-360_dataset/fruits-360/Training'
train_dir = '/content/drive/My Drive/data/sample_data/dataset/train'
#test_dir = '../input/fruits-360_dataset/fruits-360/Test'
test_dir = '/content/drive/My Drive/data/sample_data/dataset/test'

def load_dataset_and_rescale(path):
    img_datagen = ImageDataGenerator(
                                    rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)
    reszd_images = img_datagen.flow_from_directory(
                                    path,
                                    target_size = (100, 100),
                                    batch_size = 100,
                                    class_mode = 'categorical')
    return reszd_images
    
train_set = load_dataset_and_rescale(train_dir)
test_set = load_dataset_and_rescale(test_dir)
print('Loading and scaling complete!')

Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.
Loading and scaling complete!


In [25]:
# now images are reshaped to 100*100,RGB format
train_set.image_shape
test_set.image_shape

(100, 100, 3)

In [26]:
#now lets get how many classes we have.
no_of_classes = train_set.num_classes
no_of_classes

6

In [27]:
#below gives total number of data in training set
train_set.n

10901

In [28]:
train_set.image_shape

(100, 100, 3)

# **Below code is seems going in infinite loop. Lets not execute it for time being.** *italicized text*


In [0]:
# Lets try to load images as numpy with tf for plotting
#import numpy as np
#from sklearn.datasets import load_sample_image
#images_resized = tf.image.resize(train_set, [100, 100])
#images_resized = tf.image.crop_and_resize(boxes=[[0.4529,0.72,0.4664,0.7358]],image=[train_set.n,train_set.image_shape(),box_indices=train_set.batch_index[1],crop_size=[100,100])
images_resized = tf.image.resize(train_set, [100, 100])
#inputs = keras.applications.resnet50.preprocess_input(images_resized / 255)
# Load sample images
image_1 = images_resized[0]/255
image_2 = images_resized[1]/255
#china = load_sample_image("china.jpg") / 255
#flower = load_sample_image("flower.jpg") / 255
#images = np.array([china, flower])
#batch_size, height, width, channels = images.shape
images = np.array([image_1, image_2])
batch_size, height, width, channels = images.shape

# Create 2 filters
#filters = np.zeros(shape=(7, 7, channels, 2), dtype=np.float32)
#filters[:, 3, :, 0] = 1  # vertical line
#filters[3, :, :, 1] = 1  # horizontal line

#outputs = tf.nn.conv2d(images, filters, strides=1, padding="SAME")
#outputs = tf.nn.conv2d(images, filters, strides=1, padding="VALID")

plt.imshow(outputs[0, :, :, 1], cmap="gray") # plot 1st image's 2nd feature map
plt.axis("off") # Not shown in the book
plt.show()

# ResNet-34 Architecture

Lets try to create CNN with ResNet Architecture and then see its perfromance over data we have.

In [0]:

from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [0]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[100, 100, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

In [38]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_108 (Conv2D)          (None, 50, 50, 64)        9408      
_________________________________________________________________
batch_normalization_108 (Bat (None, 50, 50, 64)        256       
_________________________________________________________________
activation_3 (Activation)    (None, 50, 50, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
residual_unit_48 (ResidualUn (None, 25, 25, 64)        74240     
_________________________________________________________________
residual_unit_49 (ResidualUn (None, 25, 25, 64)        74240     
_________________________________________________________________
residual_unit_50 (ResidualUn (None, 25, 25, 64)       

In [39]:
# Compile model with parameter for loss and 
model.compile(
    loss="sparse_categorical_crossentropy", 
    optimizer="nadam", 
    metrics=["accuracy"])
print('Compiled!')

Compiled!


In [40]:
# Now train the model with checkpoint path
checkpointer = ModelCheckpoint(filepath = '/content/drive/My Drive/model/fruit_quality_check_model_ResNet_34_0001.hdf5', verbose = 1, save_best_only = True)

#history = model.fit_generator(train_set,
#       # batch_size = 32,
#        epochs=30,
#        validation_data=(test_set),
#        callbacks = [checkpointer],
#        verbose=1, shuffle=True)

history = model.fit(train_set,
        epochs=30,
        validation_data=(test_set),
        callbacks = [checkpointer],
        verbose=1, shuffle=False)

Train for 110 steps, validate for 27 steps
Epoch 1/30
  1/110 [..............................] - ETA: 23:07

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


InvalidArgumentError: ignored

# AlexNet Architecture - Seems to me

In [0]:
#Simple CNN from scratch - we are using 3 Conv layers followed by maxpooling layers.
# At the end we add dropout, flatten and some fully connected layers(Dense).

from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 2,input_shape=(100,100,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 32,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 64,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 128,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(no_of_classes,activation = 'softmax'))
model.summary()

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print('Compiled!')

In [0]:
#Lets make model checkpoint directory
!mkdir -p '/content/drive/My Drive/model'

In [0]:
# Now train the model with checkpoint path
batch_size = 32

checkpointer = ModelCheckpoint(filepath = '/content/drive/My Drive/model/fruit_quality_check_model_0002.hdf5', verbose = 1, save_best_only = True)

history = model.fit_generator(train_set,
       # batch_size = 32,
        epochs=30,
        validation_data=(test_set),
        callbacks = [checkpointer],
        verbose=2, shuffle=True)

In [0]:
# load the weights that yielded the best validation accuracy
model.load_weights('/content/drive/My Drive/model/fruit_quality_check_model_0002.hdf5')

In [0]:
# evaluate and print test accuracy
score = model.evaluate(test_set, verbose=0)
print('\n', 'Test accuracy:', score[1])
#98% accuracy !!

In [0]:
#Finally lets visualize the loss and accuracy wrt epochs

import matplotlib.pyplot as plt 
plt.figure(1)  
   
 # summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
   
 # summarize history for loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()

In [0]:
from tensorflow.keras.preprocessing import image
size=100
def predictFruitClass(ImagePath, trainedModel, class_dict):
    """
    Perform class prediction on input image and print predicted class.

    Args:
        ImagePath(str): Absolute Path to test image
        trainedModel(object): trained model from method getTrainedModel()
        DictOfClasses(dict): python dict of all image classes.

    Returns:
        Probability of predictions for each class.
    """
    x = image.load_img(ImagePath, target_size=(size,size))
    x = image.img_to_array(x)
    # for Display Only
    import matplotlib.pyplot as plt
    plt.imshow((x * 255).astype(np.uint8))
    x = np.expand_dims(x, axis=0)
    prediction_class = trainedModel.predict_classes(x, batch_size=1)
    prediction_probs = trainedModel.predict_proba(x, batch_size=1)
    print('probs:',prediction_probs)
    print('class_index:',prediction_class[0])
    for key, value in class_dict.items():
        if value == prediction_class.item():
            return key
    return None

In [0]:
!mv '/content/class_dict.npy' '/content/drive/My Drive/model'

In [0]:
class_dict = np.load('/content/drive/My Drive/model/class_dict.npy', allow_pickle=True).item()

In [0]:
image_path = '/content/drive/My Drive/data/sample_data/dataset/test/rottenoranges/Screen Shot 2018-06-12 at 11.19.56 PM.png'
single_pred = predictFruitClass(image_path,model, class_dict)
print(single_pred)

In [0]:
image_path = '/content/drive/My Drive/data/sample_data/dataset/test/rottenbanana/Screen Shot 2018-06-12 at 8.51.00 PM.png'
single_pred = predictFruitClass(image_path,model, class_dict)
print(single_pred)

In [0]:
image_path = '/content/drive/My Drive/data/sample_data/dataset/test/rottenbanana/Screen Shot 2018-06-12 at 8.58.57 PM.png'
single_pred = predictFruitClass(image_path,model, class_dict)
print(single_pred)

In [0]:
image_path = '/content/drive/My Drive/data/sample_data/dataset/test/freshapples/Screen Shot 2018-06-08 at 5.03.40 PM.png'
single_pred = predictFruitClass(image_path,model, class_dict)
print(single_pred)